In [1]:
import pandas as pd
# loyalty members by zip data received from Spencer on April 26, 2018

In [2]:
zip_dma_deduped=pd.read_csv("/home/jian/BiglotsCode/OtherInput/zipdmamapping.csv",dtype=str)
zip_dma_deduped.columns=['customer_zip_code','DMA']

In [3]:
loyalty_count_by_zip=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/Idsbyzipcode.csv",dtype=str)
loyalty_count_by_zip['customer_zip_code']=loyalty_count_by_zip['customer_zip_code'].apply(lambda x: x.zfill(5))
loyalty_count_by_zip['IDs']=loyalty_count_by_zip['IDs'].astype(int)

In [4]:
loyalty_count_by_DMA=pd.merge(zip_dma_deduped,loyalty_count_by_zip,on='customer_zip_code',how='left')
loyalty_count_by_DMA=loyalty_count_by_DMA.groupby('DMA')['IDs'].sum().to_frame()
loyalty_count_by_DMA.reset_index(inplace=True)
# loyalty_count_by_DMA.to_csv("/home/jian/Projects/Big_Lots/Loyal_member_count/sales_of_loyalty_member/Id_Counts_by_DMA.csv",index=False)



In [5]:
F25_54_by_zip=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset_2018EASI.csv",dtype=str)

F25_54_by_zip=F25_54_by_zip[['ZIP_CODE','Estimate; Female: - 25 to 29 years',
 'Estimate; Female: - 30 to 34 years',
 'Estimate; Female: - 35 to 39 years',
 'Estimate; Female: - 40 to 44 years',
 'Estimate; Female: - 45 to 49 years',
 'Estimate; Female: - 50 to 54 years']]

F25_54_by_zip_Pop_Col=[x for x in F25_54_by_zip.columns.tolist() if 'Estimate' in x]
for col in F25_54_by_zip_Pop_Col:
    F25_54_by_zip[col]=F25_54_by_zip[col].astype(float)
F25_54_by_zip['Female_25_54_Pop']=F25_54_by_zip[F25_54_by_zip_Pop_Col].sum(axis=1)
F25_54_by_zip=F25_54_by_zip[['ZIP_CODE','Female_25_54_Pop']].rename(columns={'ZIP_CODE':'customer_zip_code'})
F25_54_by_zip['customer_zip_code']=F25_54_by_zip['customer_zip_code'].apply(lambda x: x.zfill(5))

In [6]:
F25_54_by_DMA=pd.merge(F25_54_by_zip,zip_dma_deduped,on="customer_zip_code",how="left")
F25_54_by_DMA=F25_54_by_DMA.groupby(['DMA'])['Female_25_54_Pop'].sum().to_frame().reset_index()

In [7]:
output=pd.merge(F25_54_by_DMA,loyalty_count_by_DMA,on='DMA',how="outer")
output=output.rename(columns={"IDs":"Loyalty_members"})
output['Penetration']=output['Loyalty_members']/output['Female_25_54_Pop']

In [8]:
output.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/Rewards_peneration_by_DMA_JL_20180726.csv",index=False)